In [1]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils.np_utils import to_categorical

In [2]:
data = pd.read_csv('Sentiment.csv') #Looading the dataset

data = data[['text','sentiment']] # Keeping only the neccessary columns

In [3]:
data['text'] = data['text'].apply(lambda x: x.lower()) #converting to lower case
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))) #only a-z,A-Z,0-9 would be remaining in the data, else special characters are removed

In [4]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets

In [6]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values) 
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix

In [7]:
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons

In [8]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
# print(model.summary())

In [9]:
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test data split

In [11]:
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 53s - loss: 0.8207 - accuracy: 0.6467
144/144 - 3s - loss: 0.7451 - accuracy: 0.6767
0.7450950145721436
0.6767147183418274


In [12]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


## Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [13]:
model.save('sentimentAnalysis.h5') #Saving the model

In [14]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [15]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [20]:
#predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) #tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) #padding the sentence
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0] #predicting the sentence text
print(sentiment)
if sentiment == 0:
  print("Neutral")
elif sentiment < 0:
  print("Negative")
elif sentiment > 0:
  print("Positive")
else:
  print("Can not be determined")

1/1 - 0s
0
Neutral


## Apply GridSearchCV on the source code provided in the class

In [21]:
from keras.wrappers.scikit_learn import KerasClassifier #importing Keras classifier
from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

744/744 - 108s - loss: 0.8264 - accuracy: 0.6457
186/186 - 3s - loss: 0.7517 - accuracy: 0.6729
744/744 - 103s - loss: 0.8241 - accuracy: 0.6474
186/186 - 3s - loss: 0.7517 - accuracy: 0.6832
744/744 - 103s - loss: 0.8248 - accuracy: 0.6465
186/186 - 3s - loss: 0.7615 - accuracy: 0.6649
744/744 - 101s - loss: 0.8232 - accuracy: 0.6477
186/186 - 3s - loss: 0.7584 - accuracy: 0.6744
744/744 - 110s - loss: 0.8207 - accuracy: 0.6426
186/186 - 4s - loss: 0.7927 - accuracy: 0.6539
Epoch 1/2
744/744 - 106s - loss: 0.8252 - accuracy: 0.6484
Epoch 2/2
744/744 - 102s - loss: 0.6775 - accuracy: 0.7135
186/186 - 3s - loss: 0.7590 - accuracy: 0.6434
Epoch 1/2
744/744 - 126s - loss: 0.8287 - accuracy: 0.6447
Epoch 2/2
744/744 - 108s - loss: 0.6852 - accuracy: 0.7096
186/186 - 3s - loss: 0.7468 - accuracy: 0.6719
Epoch 1/2
744/744 - 109s - loss: 0.8343 - accuracy: 0.6419
Epoch 2/2
744/744 - 98s - loss: 0.6859 - accuracy: 0.7086
186/186 - 4s - loss: 0.7626 - accuracy: 0.6848
Epoch 1/2
744/744 - 114s -